In [89]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import datetime

In [90]:
# Import the data from the json file
# Data is in format {},{},{},... and in a text file

# Read the data from the file
with open('../../Dataset/build_dataset.txt') as f:
    data = f.readlines()

# Convert the data to a list of dictionaries
data = [json.loads(x.strip()) for x in data]

In [91]:
def get_readers_by_document(data, doc_uuid):
    all_readers = []
    for record in data:
        if 'subject_doc_id' in record and  record['subject_doc_id'] == doc_uuid and 'visitor_uuid' in record:
            all_readers.append(record['visitor_uuid'])
    return all_readers


In [92]:
def get_document_by_readers(data, visitor_uuid):
    all_docs = []
    for record in data:
        if 'visitor_uuid' in record and record['visitor_uuid'] == visitor_uuid and 'subject_doc_id' in record:
            
                all_docs.append(record['subject_doc_id'])
    return all_docs


In [93]:
def also_likes(data, doc_uuid, sorting_function=None):

    if sorting_function is None:
        sorting_function = lambda x: x[1]

    all_readers = get_readers_by_document(data, doc_uuid)
    unique_readers = set(all_readers)

    liked_documents = {}

    for reader in unique_readers:
            docs = get_document_by_readers(data, reader)
            unique_docs = set(docs)
            for doc in unique_docs:
                if doc != doc_uuid:
                    liked_documents[doc] = liked_documents.get(doc, 0) + 1

    sorted_docs = sorted(liked_documents.items(), key=sorting_function, reverse=True)
    return [doc[0] for doc in sorted_docs]

In [94]:
def sort_by_readers(item):
    return item[1]

In [95]:
doc_uuid = "100713205147-2ee05a98f1794324952eea5ca678c026"  

# Get the top 10 'also likes' documents
top_liked_docs = also_likes(data, doc_uuid, sort_by_readers)
for doc in top_liked_docs[:10]:
    print(doc)